In [1]:
import picamera
import picamera.array

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import time
import numpy as np
import cv2
from PIL import Image
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
import cv2
import numpy as np
import os


Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [4]:
p = 'dataset'
TRAINING_DATA = []  # Example array to be trained
TRAINING_LABELS = []  # Label array
NUM_CLASSES = 0
count = [0, 0]

NUM_CLASSES = 2
LEARNING_RATE = 0.0001
EPOCHS = 10
DENSE_UNITS = 100


In [5]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os

#image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#display(image)

In [6]:
# class0 = 'dataset/0'
# class1 = 'dataset/1'

In [7]:
def get_image(path):
    print("get image called")
    global NUM_CLASSES
    image_path = [os.path.join(path, f) for f in os.listdir(path)]
    classes = [p.split('/')[1] for p in image_path]
    NUM_CLASSES = len(classes)

    print(NUM_CLASSES)
    print(classes)
#     classes = classes[1:]
#     print(classes)
    for cls in classes:
        pp = path + "/" + cls
        print("class={}".format(pp))
        image_path = [ os.path.join(pp, f) for f in os.listdir(pp) if f.endswith('.jpg') ]
        print(image_path)
        print(len(image_path))
        for img in image_path:
            print("path={}".format(img))
            add_example(prepare_frame(cv2.imread(img)), int(cls))

In [8]:
def add_example(example, label):  # add examples to training data set
    encoded_y = keras.utils.np_utils.to_categorical(label, num_classes=NUM_CLASSES)  # make one-hot
    TRAINING_LABELS.append(encoded_y)
    TRAINING_DATA.append(example[0])
    print('> add example for label %d' % label)
    count[label] += 1


def prepare_frame(frame):
    img = Image.fromarray(frame, 'RGB')
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array_extended = np.expand_dims(img_array, axis=0).astype('float32')
    processed = keras.applications.mobilenet.preprocess_input(img_array_extended)
    return processed

In [9]:
get_image("dataset")

get image called
2
['1', '0']
class=dataset/1
['dataset/1/1574682713.6998224.jpg', 'dataset/1/1574682720.8157759.jpg', 'dataset/1/1574682733.5838559.jpg', 'dataset/1/1574682715.5508034.jpg', 'dataset/1/1574682718.302046.jpg', 'dataset/1/1574682723.8483438.jpg', 'dataset/1/1574682714.632949.jpg', 'dataset/1/1574682725.6288648.jpg', 'dataset/1/1574682717.436506.jpg', 'dataset/1/1574682712.8491585.jpg', 'dataset/1/1574682716.4929962.jpg', 'dataset/1/1574682719.899265.jpg', 'dataset/1/1574682728.2821305.jpg', 'dataset/1/1574682727.1334388.jpg', 'dataset/1/1574682721.750257.jpg', 'dataset/1/1574682729.1823766.jpg', 'dataset/1/1574682724.7404063.jpg', 'dataset/1/1574682732.626225.jpg', 'dataset/1/1574682722.6861837.jpg']
19
path=dataset/1/1574682713.6998224.jpg
> add example for label 1
path=dataset/1/1574682720.8157759.jpg
> add example for label 1
path=dataset/1/1574682733.5838559.jpg
> add example for label 1
path=dataset/1/1574682715.5508034.jpg
> add example for label 1
path=dataset/1/1

In [10]:

def load_model():
    mobilenet = keras.applications.mobilenet.MobileNet()
    flatten = Flatten(input_shape=(7, 7, 1024))(mobilenet.get_layer('conv_pw_13_relu').output)
    fc1 = Dense(DENSE_UNITS, activation='relu')(flatten)
    fc2 = Dense(NUM_CLASSES)(flatten)
    output = Activation('softmax')(fc2)
    model = Model(mobilenet.input, output)
    # make all layers untrainable by freezing weights (except for last two layers)
    for l, layer in enumerate(model.layers[:-3]):
        layer.trainable = False
    # ensure the last layer is trainable/not frozen
    for l, layer in enumerate(model.layers[-3:]):
        layer.trainable = True
    return model

In [11]:

model = load_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
model

In [ ]:
model.fit(np.array(TRAINING_DATA), np.array(TRAINING_LABELS), epochs=EPOCHS, batch_size=8)


Epoch 1/10
42/42 [==============================] - 31s 733ms/step - loss: 11.8761 - accuracy: 0.5952
Epoch 2/10
42/42 [==============================] - 28s 659ms/step - loss: 2.5003 - accuracy: 0.7619
Epoch 3/10
42/42 [==============================] - 28s 657ms/step - loss: 1.9948 - accuracy: 0.7857
Epoch 4/10
 8/42 [====>.........................] - ETA: 22s - loss: 0.0493 - accuracy: 1.0000

In [ ]:
model.predict

In [ ]:
with picamera.PiCamera() as camera:
    with picamera.array.PiRGBArray(camera) as output:
        camera.capture(output, 'rgb')
        print('Captured %dx%d image' % (output.array.shape[1], output.array.shape[0]))
        frame = output.array
        processed_image = prepare_frame(frame)  # prepare frame
        prediction = model.predict(processed_image)
        result = np.argmax(prediction)  # imagenet_utils.decode_predictions(prediction)
        print('predict class= %d' % result)
